## Setup

In [4]:
# Uncomment if you need to install 
# !pip install random_survival_forest

In [5]:
import pandas as pd
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from random_survival_forest import RandomSurvivalForest, concordance_index

np.random.seed(1337)

In [6]:
D = sio.loadmat('data.mat')
#dict_keys(['__header__', '__version__', '__globals__', 'Censored', 'Integ_Symb_Types', 'Integ_Symbs', 'Integ_X', 'Integ_X_raw', 'Patients', 'Subtypes', 'Survival'])

T = np.asarray([t[0] for t in D['Survival']]).astype('float32')
O = 1 - np.asarray([c[0] for c in D['Censored']]).astype('int32')
X = D['Integ_X_raw'].astype('float32')
X_headers = list(D['Integ_Symbs'])

df = pd.DataFrame(data=X, columns=X_headers)
df['Survival'] = T
df['Censored'] = O
print(df.shape)
df.head()

(560, 401)


,age_at_initial_pathologic_diagnosis_Clinical,gender-Is-male_Clinical,histological_type-Is-oligoastrocytoma_Clinical,histological_type-Is-astrocytoma_Clinical,histological_type-Is-oligodendroglioma_Clinical,histological_type-Is-glioblastoma multiforme (gbm)_Clinical,histological_type-Is-treated primary gbm_Clinical,histological_type-Is-untreated primary (de novo) gbm_Clinical,radiation_therapy-Is-yes_Clinical,ACADS_Mut,AOX1_Mut,ARID1A_Mut,ARID2_Mut,ATF7IP2_Mut,ATRX_Mut,BCOR_Mut,BRAF_Mut,C10orf76_Mut,CD1D_Mut,CD209_Mut,CD44_Mut,CDKN2A_Mut,CIC_Mut,CMA1_Mut,CNOT1_Mut,CPEB4_Mut,CREBZF_Mut,CYP11A1_Mut,DDX5_Mut,DLC1_Mut,DLX6_Mut,DNMT3A_Mut,DOCK5_Mut,EEF1A1_Mut,EGFR_Mut,EMG1_Mut,EPS8L1_Mut,ESR2_Mut,FAM123C_Mut,FAM126B_Mut,...,STMN1_Protein,SYK_Protein,WWTR1_Protein,TFRC_Protein,TIGAR_Protein,TSC1_Protein,TGM2_Protein,TSC2_Protein,TSC2_Protein,KDR_Protein,XRCC1_Protein,YAP1_Protein,YAP1_Protein,YBX1_Protein,YBX1_Protein,CTNNB1_Protein,JUN_Protein,KIT_Protein,MET_Protein,MYC_Protein,BIRC2_Protein,EEF2_Protein,EEF2K_Protein,EIF4E_Protein,EIF4G1_Protein,MTOR_Protein,MTOR_Protein,CDKN1A_Protein,CDKN1B_Protein,CDKN1B_Protein,CDKN1B_Protein,MAPK14_Protein,TP53_Protein,SQSTM1_Protein,RPS6KB1_Protein,RPS6KB1_Protein,RPS6KA1_Protein,RPS6KA1_Protein,Survival,Censored
0,50.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.231969,1.236948,0.568017,-0.189747,0.101076,-0.207325,0.303399,-0.144826,-0.364267,0.592570,-0.016465,0.223397,-0.554451,-0.646095,-0.177982,-1.286851,-0.101540,-0.783198,0.353918,0.215715,-0.201761,-0.108027,-0.336988,-0.107154,-0.516484,-0.394360,0.166735,0.418603,-0.032919,0.215703,0.161241,-0.097997,0.108144,0.661336,-0.111310,0.474086,0.377963,0.057217,144.0,1
1,57.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.057435,-1.398708,0.602139,-1.620308,-0.214998,-1.117199,0.626779,-1.945139,-1.310087,-0.860510,0.242296,1.593988,1.362545,-0.001860,-0.162358,-0.712493,0.069898,1.087936,0.449773,0.276249,-0.192023,-0.661946,-0.711138,-0.244208,-0.965930,-2.255737,-0.747773,0.698555,0.593962,0.397792,0.740407,-0.118854,0.269457,0.116623,-0.573753,-0.587442,-0.523980,0.000000,393.0,1
2,53.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.065149,-0.409672,-0.089685,-0.159985,-0.311739,-0.143713,0.053068,-0.467774,-0.029763,-0.434648,0.193044,0.277397,-0.184019,-0.521961,0.031076,-0.027812,0.255605,4.742699,-0.199033,-0.318494,0.015981,-0.150997,-0.693784,-0.001438,-0.262349,-0.207268,0.036444,-0.278100,0.175505,-0.118972,0.049271,-0.204173,0.426592,-0.379803,-0.158300,0.100670,-0.481545,0.098332,470.0,0
3,86.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.320259,-0.116542,-0.122051,0.260292,-0.082233,0.054077,-0.023269,0.651192,0.698792,0.075816,-0.050477,-0.147755,-0.376620,0.175936,0.324163,0.781109,0.187001,-0.063293,-0.114954,-0.110859,-0.054756,-0.173451,0.271884,-0.063351,0.187555,0.386391,0.340228,0.131084,-0.213529,-0.158576,-0.126167,-0.064162,-0.223155,-0.188487,0.292586,-0.044545,-0.018513,0.085689,211.0,1
4,66.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.095470,-1.804135,-0.496847,-1.849024,-0.611730,-1.328199,-0.229339,-2.545047,-0.308216,-0.933520,0.485868,0.088308,0.079703,-0.835898,-0.083564,0.317362,0.891561,0.022763,0.105622,0.773883,-1.002407,0.501054,0.079552,-0.248789,-0.837197,-1.356546,-0.002290,-0.354671,1.474617,-0.730698,0.262911,-0.594671,0.878369,0.137746,-1.202437,0.260542,-0.026632,1.196287,691.0,1


In [7]:
y = df.iloc[:,-2:]
X = df.iloc[:,:-2]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(y_train.shape))
print("shape of Y Test :"+str(y_test.shape))

shape of X Train :(420, 399)
shape of X Test :(140, 399)
shape of Y Train :(420, 2)
shape of Y Test :(140, 2)


## Establish a baseline using Logistic Regression


In [ ]:
#logistic Regression

#logistic Regression
y = df[["Survival", "Censored"]]
#y = df[["Survival"]]
y["Survival"] = y["Survival"].values.astype(int)
X = df.drop(columns=["Survival", "Censored"])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train["Survival"])
y_pred=logreg.predict(X_test)
#cnf_matrix = metrics.confusion_matrix(y_test["Survival"], y_pred["Censored"])


#c_index = concordance_index(logreg, y_test, y_pred)
#print('C-index: {:.2f}'.format(c_index))
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#print("Precision:",metrics.precision_score(y_test, y_pred))
#print("Recall:",metrics.recall_score(y_test, y_pred))
c_val = concordance_index(y_time=y_test["Survival"], y_pred=y_pred, y_event=y_test["Censored"])
print('C-index: {:.2f}'.format(c_val))


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


C-index: 0.38


## Establish a baseline using Support Vector Machine (SVM)

In [ ]:
# Uncomment if you need to install 
#!pip install pysurvival

In [9]:
from pysurvival.models.svm import LinearSVMModel
from pysurvival.utils.metrics import concordance_index

In [10]:
svm_model = LinearSVMModel()
svm_model.fit(X=X_train, T=y_train["Survival"], E=y_train["Censored"], init_method='he_normal', lr = 0.5,  
              tol = 1e-3,  l2_reg = 1e-3, verbose = False)


LinearSVMModel

In [11]:
c_index = concordance_index(svm_model, X=X_test, T=y_test["Survival"], E=y_test["Censored"])
print('C-index: {:.2f}'.format(c_index))

C-index: 0.66


## Establish a baseline using Random Survival Forest (RSF)

In [ ]:
from random_survival_forest import RandomSurvivalForest, concordance_index

y = df[["Survival", "Censored"]]
y["Survival"] = y["Survival"].values.astype(int)
X = df.drop(columns=["Survival", "Censored"])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)


rsf = RandomSurvivalForest(n_estimators=10)
rsf = rsf.fit(X_train, y_train)
y_pred = rsf.predict(X_test)
c_val = concordance_index(y_time=y_test["Survival"], y_pred=y_pred, y_event=y_test["Censored"])

In [ ]:
print("C-index", round(c_val, 3))

## Neural Network SurvivalNet Reimplementation

In [15]:
# Uncomment if you need to install 
#!pip install keras_tuner

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
import keras_tuner as kt

tf.random.set_seed(1337)

In [16]:
class partial_log_likelihood(keras.losses.Loss):
    def __init__(self, name="partial_log_likelihood"):
        super().__init__(name=name)
    def call(self, y_true, y_pred):
        censored = y_true[:,1]
        y_true = y_true[:,0]
        y_pred = tf.reshape(y_pred, [-1])
        sorted_survival, sorted_indices = tf.math.top_k(y_true, len(y_true), True) 
        sorted_censored = keras.backend.gather(censored, sorted_indices)
        sorted_preds = keras.backend.gather(y_pred, sorted_indices)
        exp = keras.backend.exp(sorted_preds)
        sums = keras.backend.cumsum(exp) + tf.convert_to_tensor(1.0)
        logsums = keras.backend.log(sums)
        
        return keras.backend.sum(sorted_censored * sorted_preds - sorted_censored*logsums) * -1

In [17]:
# Model outline with hyperparameter ranges/choices to be searched by BayesOpt tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int("units1", min_value=32, max_value=512, step=32), activation='relu', input_dim=len(X_train.columns)))
    model.add(Dropout(hp.Float("drop_rate", min_value=0.0, max_value=0.2, step=0.1)))
    model.add(Dense(hp.Int("units2", min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dropout(hp.Float("drop_rate", min_value=0.0, max_value=0.2, step=0.1)))
    model.add(Dense(hp.Int("units3", min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])),loss=partial_log_likelihood(), run_eagerly=True)
    return model

In [ ]:
# Run Bayesian Optimization on the NN survival model to find best hyperparameters
tuner = kt.BayesianOptimization(build_model, objective="loss", max_trials=20, executions_per_trial=1, directory="./bayes_opt", project_name="logs")
y_train.loc[:,"Censored"] = np.array(y_train.loc[:,"Censored"].values).astype(np.float32)
tuner.search(X_train, y_train.values, batch_size=32, epochs=50)

Trial 20 Complete [00h 00m 24s]
loss: 10.101755142211914

Best loss So Far: 4.774790287017822
Total elapsed time: 00h 09m 26s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Rebuild model with best hyperparameters selected by bayesian optimization
best_hp = tuner.get_best_hyperparameters()[0]
model = tuner.hypermodel.build(best_hp)

In [ ]:
# Retrain best model
model.fit(X_train, y_train.values,batch_size=32, epochs=50, verbose=False)

In [ ]:
print(f'c-index: {concordance_index(y_time=y_test["Survival"], y_pred=model.predict(X_test), y_event=y_test["Censored"])}')

c-index: 0.8695114409400123
